In [10]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import keras

Using TensorFlow backend.


In [11]:
# Load in training and test set data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train = train.dropna()
test_ids = test['id']
test = test.drop(['id'], axis=1)

# Replace NaN values with 0
# test = test.fillna(0)
test = test.fillna(test.mean())

In [3]:
# Preview training set data
train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1
5,5,3843.6,3844.2,12.0,17.0,29.0,-5,3843.8,3843.4,3843.2,...,6,1,2,17,1,12,15,10,3,0


In [6]:
# Preview test set data
test.head()

,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,bid4,...,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol
0,5178.4,5178.3,1.000000,11.000000,12.0,-10,5178.2,5178.0,5177.8,5177.6,...,3,16,3,1,1,1,4,1,5,2
1,5133.0,5132.9,1.000000,0.000000,1.0,1,5132.8,5132.6,5132.4,5132.2,...,1,2,2,1,10,4,8,1,1,1
2,5177.4,5178.0,2.000000,0.000000,2.0,2,5177.2,5176.8,5176.6,5176.4,...,8,18,1,1,6,7,4,1,2,5
3,5093.6,5093.9,0.000000,2.000000,2.0,-1,5093.2,5093.0,5092.8,5092.2,...,8,5,4,3,1,1,1,11,6,2
4,5189.2,5189.2,1.444629,1.936579,0.0,0,5188.8,5188.6,5188.4,5188.2,...,1,5,1,3,3,3,2,1,3,4


In [7]:
print(train.shape)
print(test.shape)

(419920, 28)
(191859, 26)


In [12]:
train_X, test_X, train_y, test_y = train_test_split(train.loc[:,'last_price':'ask5vol'], 
                                                    train.loc[:,'y'], test_size=0.2, random_state=155155155)

In [10]:
print(train_X.shape)

(335936, 26)


In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.optimizers import SGD
from sklearn.utils import compute_class_weight

In [12]:
model = Sequential()
model.add(Dense(8, input_dim=26, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.001, decay=1e-6, momentum=1.9)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

classWeight = compute_class_weight('balanced', np.array([0,1]), train_y) 
classWeight = dict(enumerate(classWeight))

model.fit(train_X, train_y, epochs=10, batch_size=256, class_weight=classWeight)

W0212 19:50:07.493729 4629472704 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0212 19:50:07.506623 4629472704 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0212 19:50:07.508790 4629472704 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0212 19:50:07.559781 4629472704 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0212 19:50:07.564022 4629472704 dep

Epoch 1/10
335936/335936 [==============================] - 1s 4us/step - loss: 8.0590 - acc: 0.6289
Epoch 2/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 3/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 4/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 5/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 6/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 7/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 8/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 9/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - acc: 0.6289
Epoch 10/10
335936/335936 [==============================] - 1s 3us/step - loss: 8.0590 - a

In [29]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(20, 10), 
                   random_state=1, max_iter=500)

nn.fit(train_X, train_y)
# nn.fit(train.loc[:,'last_price':'ask5vol'], train.loc[:,'y'])

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [30]:
print(1 - nn.score(train_X,train_y))
print(1 - nn.score(test_X,test_y))

0.37113319203657835
0.3720113354924748


In [22]:
# Predict probabilities on validation set
pred = nn.predict_proba(test.to_numpy())

print(pred)

# Probabilities of being labeled 1
pred = pd.DataFrame(pred[:,1], columns=['Predicted'])
pred.insert(0, 'id', test_ids)
pred.to_csv("nn_probs_submission.csv", index=False)

[[1.00000000e+00 1.97750203e-53]
 [1.00000000e+00 1.34800452e-52]
 [1.00000000e+00 6.59579648e-53]
 ...
 [1.00000000e+00 1.97343445e-53]
 [1.00000000e+00 7.25566338e-53]
 [1.00000000e+00 1.24787276e-53]]
